LEGB (local, enclosed, global, built-in) functions do not apply when we are making new name bindings

In [1]:
message = 'global'

def enclosing():
    message = 'enclosing'

    def local():
        message = 'local'

When we assign to message in the function local, we are creating a new name binding in that function's scope from the name message to the string "local".  We are not rebinding either of the other two message variables in the code.

In [3]:
message = 'global'

def enclosing():
    message = 'enclosing'

    def local():
        message = 'local'

    print('enclosing message:', message)
    local()
    print('enclosing message:', message)

print('global message:', message)
enclosing()
print('global message:', message)

global message: global
enclosing message: enclosing
enclosing message: enclosing
global message: global


If we wanted the function local() to odify the global binding for message rather than create a new one, we could use the global keywork to introduce the global message binding into local:

In [4]:
message = 'global'

def enclosing():
    message = 'enclosing'

    def local():
        global message
        message = 'local'

    print('enclosing message:', message)
    local()
    print('enclosing message:', message)

print('global message:', message)
enclosing()
print('global message:', message)

global message: global
enclosing message: enclosing
enclosing message: enclosing
global message: local


As evidenced from the code above the module-level binding of the message is changed when local is called.

So how an we make the function local() modify the binding for message defined in the function enclosing?  Use of the keywork nonloal which inserts a name biding from an enclosing namespace into the local namespace.  In other words, nonlocal searches the enclosing namespaces from innermost to outermost for the name you give it.  Once it finds a match, that binding is introduced into the scope where nonlocal was invoked:

In [5]:
message = 'global'

def enclosing():
    message = 'enclosing'

    def local():
        nonlocal message
        message = 'local'

    print('enclosing message:', message)
    local()
    print('enclosing message:', message)

print('global message:', message)
enclosing()
print('global message:', message)

global message: global
enclosing message: enclosing
enclosing message: local
global message: global


Note that you can trhow an error when no matching enclosing binding exists.  Python will raise a SyntaxError:

In [6]:
message = 'global'

def enclosing():
    message = 'enclosing'

    def local():
        nonlocal no_such_name
        message = 'local'

    print('enclosing message:', message)
    local()
    print('enclosing message:', message)

print('global message:', message)
enclosing()
print('global message:', message)

SyntaxError: no binding for nonlocal 'no_such_name' found (<ipython-input-6-a45d3c44bdb5>, line 7)

In this example the make_timer() function returns a new function.  Each time you call this new function, it returns the elapsed time since the last time you called it.

In [7]:
import time

def make_timer():
    last_called = None # Never

    def elasped():
        nonlocal last_called
        now = time.time()
        if last_called is None:
            last_called = now
            return None
        result = now - last_called
        last_called = now
        return result

    return elasped

In [8]:
t = make_timer()

In [9]:
t()

In [10]:
t()

4.85308313369751

In [17]:
t()

2.7054452896118164

In [18]:
t()

9.703350067138672

As you can see the first time you invoke t it returns nothing.  After that, it returns the amount of time since the last invocation.  Every time you call make_timer(), it creates a new local variable named last_called.  It then defines a local function called elapsed() which uses the nonlocal keyword to insert make_timers()'s binding of last_called into its local scope.  The inner elapsed() elaped function then uses the last_called binding to keep track of the last time it was called.  In other words, elapsed() uses nonlocal to refer to a name binding which will exist across multiple calls to be elapsed().  In this way, elapsed is using nonlocal to create a form of persistent storage.

Note that each call to make_timer() creates a new, independent binding of last_called() as well as a new definition of elapsed().  This means that each call to make_timer() creates a new, independent timer object:

In [19]:
t1 = make_timer()
t2 = make_timer()

In [20]:
t1()
t1()

3.147125244140625e-05

In [21]:
t2()
t2()

3.075599670410156e-05

In [22]:
t2()

7.906819820404053

In [23]:
t2()

6.852389574050903

In [24]:
t1()

47.58114671707153

In [25]:
t2()

28.4842689037323